In [1]:
import pandas as pd

## Load the unlabelled dataset ##

In [ ]:
df = pd.read_csv('data/from-azure/AllData.csv')

In [ ]:
features_cols = ['GridRows', 'GridColumns', 'NumOfAgents', 'NumOfObstacles', 'BranchingFactor', 'ObstacleDensity',
            'AvgDistanceToGoal', 'MaxDistanceToGoal', 'MinDistanceToGoal', 'AvgStartDistances', 'AvgGoalDistances',
            'PointsAtSPRatio']
runtime_cols = list(df.filter(like="Runtime"))
relev_cols = runtime_cols + features_cols + ['Y']

In [ ]:
runtime_cols

## Create Y Column with argmax on runtime_cols ##

In [ ]:
Y = df[runtime_cols].idxmin(axis=1)
df['Y'] = Y

In [ ]:
def runtime_to_success(col):
    splitted = col.split()
    splitted[-1] = "Success"
    return " ".join(splitted)

runtime_to_success("Y Runtime")

In [ ]:
df['Y Success'] = df.apply(lambda x:x[runtime_to_success(x['Y'])], axis=1)

In [ ]:
df['Y Runtime'] = df.apply(lambda x:x[x['Y']], axis=1)

In [ ]:
df['Y'].value_counts()

## Add Sparsity feature 

In [ ]:
df['GridSize'] = df['GridRows'] * df['GridColumns']

In [ ]:
df['Sparsity'] = df.apply(lambda x: x['NumOfAgents']/(x['GridSize']-x['NumOfObstacles']),axis=1)

In [ ]:
features_cols.append('GridSize')
features_cols.append('Sparsity')


In [71]:
features_cols.append('Y')
df[features_cols].corr()

,GridRows,GridColumns,NumOfAgents,NumOfObstacles,BranchingFactor,ObstacleDensity,AvgDistanceToGoal,MaxDistanceToGoal,MinDistanceToGoal,AvgStartDistances,AvgGoalDistances,PointsAtSPRatio,GridSize,Sparsity
GridRows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GridColumns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NumOfAgents,NaN,NaN,1.000000e+00,NaN,3.166183e-01,-6.773119e-16,1.499232e-01,4.811157e-01,-0.417603,6.526064e-01,6.980675e-01,9.391103e-01,NaN,1.000000e+00
NumOfObstacles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BranchingFactor,NaN,NaN,3.166183e-01,NaN,1.000000e+00,2.453595e-16,5.384168e-02,1.846006e-01,-0.097271,7.674998e-02,1.071488e-01,1.684845e-01,NaN,3.166183e-01
ObstacleDensity,NaN,NaN,-6.773119e-16,NaN,2.453595e-16,1.000000e+00,-2.252270e-16,6.440290e-16,0.000000,1.817104e-15,-1.984054e-16,1.886224e-16,NaN,6.593109e-16
AvgDistanceToGoal,NaN,NaN,1.499232e-01,NaN,5.384168e-02,-2.252270e-16,1.000000e+00,5.148223e-01,0.205369,2.734999e-01,1.969052e-01,3.116087e-01,NaN,1.499232e-01
MaxDistanceToGoal,NaN,NaN,4.811157e-01,NaN,1.846006e-01,6.440290e-16,5.148223e-01,1.000000e+00,-0.096289,6.382731e-01,4.535112e-01,6.004857e-01,NaN,4.811157e-01
MinDistanceToGoal,NaN,NaN,-4.176027e-01,NaN,-9.727106e-02,0.000000e+00,2.053689e-01,-9.628908e-02,1.000000,-2.515828e-01,-3.147113e-01,-4.061105e-01,NaN,-4.176027e-01
AvgStartDistances,NaN,NaN,6.526064e-01,NaN,7.674998e-02,1.817104e-15,2.734999e-01,6.382731e-01,-0.251583,1.000000e+00,6.505115e-01,7.474735e-01,NaN,6.526064e-01


## Save labelled data to file

In [72]:
#df.to_csv("data/from-azure/kiva-labelled.csv")